## Uvod i Analiza podataka (EDA)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Postavljanje stila za grafikone
sns.set_theme(style="whitegrid")

# Učitavanje podataka (korišćenje relativne putanje za GitHub)
try:
    df = pd.read_csv('products_clean.csv')
    # Čišćenje praznih polja koja uzrokuju greške u TF-IDF
    df = df.dropna(subset=['Product Title', 'Category Label'])
    print(f"✅ Dataset uspešno učitan. Broj artikala nakon čišćenja: {len(df)}")
except FileNotFoundError:
    print("❌ Greška: 'products_clean.csv' nije pronađen u folderu!")

# Vizuelizacija distribucije kategorija
plt.figure(figsize=(15, 8))
order = df['Category Label'].value_counts().index
sns.countplot(data=df, y='Category Label', order=order, palette='magma')
plt.title('Analiza podataka: Distribucija proizvoda po kategorijama', fontsize=16)
plt.xlabel('Broj proizvoda')
plt.ylabel('Kategorija')
plt.show()

## Uporedni trening više modela

In [ ]:
# Priprema podataka
X = df['Product Title'].astype(str).str.lower()
y = df['Category Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli za testiranje
models = {
    'Linear SVC': LinearSVC(random_state=42),
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000)
}

results = {}

print("🚀 Treniranje i upoređivanje modela...\n")

for name, clf in models.items():
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1, 3))),
        ('clf', clf)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name:>20}: {acc:.4f}")

# Grafički prikaz poređenja modela
plt.figure(figsize=(10, 5))
sns.barplot(x=list(results.keys()), y=list(results.values()), palette='coolwarm')
plt.ylim(0.9, 1.0)
plt.title('Uporedni trening: Preciznost modela (Accuracy Score)')
plt.ylabel('Tačnost')
plt.show()

## Detaljna evaluacija najboljeg modela (Matrica konfuzije)

In [ ]:
# Treniramo pobednički model za detaljnu analizu
best_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1, 3))),
    ('clf', LinearSVC(random_state=42))
])

best_pipeline.fit(X_train, y_train)
y_pred = best_pipeline.predict(X_test)

# Generisanje matrice konfuzije
plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=best_pipeline.classes_, 
            yticklabels=best_pipeline.classes_)

plt.title('Matrica konfuzije za Linear SVC model', fontsize=15)
plt.ylabel('Stvarna kategorija')
plt.xlabel('Predviđena kategorija')
plt.show()

# Ispis detaljnog izveštaja (Precision, Recall, F1-Score)
print("\n📋 Detaljan izveštaj klasifikacije:")
print(classification_report(y_test, y_pred))

## Zaključak

### 🏁 Zaključak projekta

1.  **Analiza podataka:** Dataset je inicijalno sadržao NaN vrednosti u nazivima proizvoda koje su očišćene kako bi model mogao ispravno da radi. Distribucija kategorija pokazuje da su podaci relativno dobro balansirani.
2.  **Poređenje modela:** Testirali smo tri algoritma:
    * **Linear SVC** je ostvario najbolji rezultat (~95.7%).
    * **Logistic Regression** je bio blizu, dok je **Naive Bayes** pokazao nešto slabije rezultate na trigramima.
3.  **Optimizacija:** Korišćenje `ngram_range=(1, 3)` (unigrami, bigrami i trigrami) bilo je ključno za prepoznavanje specifičnih naziva modela bele tehnike.
4.  **Finalno rešenje:** Iako je Linear SVC izuzetno precizan, za produkcionu skriptu `predict_category.py` implementiran je **Hibridni sistem** koji kombinuje ovaj model sa "rule-based" logikom kako bi se rešili specifični izazovi sa brendovima kao što je Bosch.